In [1]:
import numpy as np
import strawberryfields as sf
import tensorflow as tf
import random

In [5]:
tf_displacement_magnitude = tf.Variable(0.1)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
steps = 500

batch_size = 10
threshold = 0.5

for step in range(steps):
    batch = [1 if random.random() > threshold else -1 for _ in range(batch_size)]
    alpha_val = 0.7 * np.array(batch)
    
    p_1 = batch.count(1)/batch_size
    p_0 = 1 - p_1
        
    eng = sf.Engine(backend="tf", backend_options={
        "cutoff_dim": 7,
        "batch_size": batch_size,
    })
    
    circuit = sf.Program(1)

    displacement_magnitude = circuit.params("displacement_magnitude")
    
    alpha = circuit.params("alpha")

    with circuit.context as q:
        sf.ops.Dgate(alpha, 0.0) | q[0]
        sf.ops.Dgate(displacement_magnitude, 0.0) | q[0]
        
    with tf.GradientTape() as tape:
        results = eng.run(circuit, args={
            "displacement_magnitude": tf_displacement_magnitude,
            "alpha": alpha_val
        })
        
        # get the probability of |0>
        p_zero = results.state.fock_prob([0])
        # print(p_zero)
        
        # get the porbability of anything by |0>
        p_one = 1 - p_zero
        
        loss = 0
        
        # p(a|1) = [p(1|a) p(1)] / p(a)
        # p(-a|0) = [p(0|-a) p(0)] / p(-a)
        
        for i, mult in enumerate(batch):
            if mult == 1:
                loss += p_one[i]
            else:
                loss += p_zero[i]
                
        loss /= batch_size
        # print(f'loss: {loss}')
        
    gradients = tape.gradient(loss, [tf_displacement_magnitude])
    opt.apply_gradients(zip(gradients, [tf_displacement_magnitude]))
    
    if (step + 1) % 10 == 0:
        print("Learned displacement value at step {}: {}".format(step+1, tf_displacement_magnitude.numpy()))

Learned displacement value at step 10: -8.555687963962555e-05
Learned displacement value at step 20: -0.10022669285535812
Learned displacement value at step 30: -0.1994660496711731
Learned displacement value at step 40: -0.29590293765068054
Learned displacement value at step 50: -0.38713476061820984
Learned displacement value at step 60: -0.4713663160800934
Learned displacement value at step 70: -0.5458700060844421
Learned displacement value at step 80: -0.6106246113777161
Learned displacement value at step 90: -0.663436770439148
Learned displacement value at step 100: -0.7067794799804688
Learned displacement value at step 110: -0.7424363493919373
Learned displacement value at step 120: -0.7681866884231567
Learned displacement value at step 130: -0.7864252924919128
Learned displacement value at step 140: -0.7998858690261841
Learned displacement value at step 150: -0.8065536022186279
Learned displacement value at step 160: -0.8140542507171631
Learned displacement value at step 170: -0.8